<img src="slides/6.png">

## ローンデータセット

ローンのデータセットを決定木により分析。

In [ ]:
import pandas as pd
import numpy as np
loan = pd.read_csv('data/loan_data.csv')

# 使用しない列を削除
df = loan.drop(['Unnamed: 0', 'status', 'loan_amnt', 'term', 'annual_inc', 'dti',
                  'revol_bal', 'revol_util', 'purpose', 'home_ownership',
                  'delinq_2yrs_zero', 'pub_rec_zero', 'open_acc', 'grade'], axis=1)
# 確認
df

### ダミー変数化

カテゴリカル変数について pos_label で正例ラベルを指定せずに、正確度、適合度、再現率を計算できるようにするため、目的変数もダミー変数化

- ダミー変数化すると、決定木の描画時に文字列のクラスラベルを指定する必要が出てくる

In [ ]:
# ダミー変数化
df['outcome'] = df['outcome'].map(lambda x: 1 if x == 'default' else 0)
df = pd.get_dummies(data=df)
# 確認
# - 0行: default -> 1
# - 45341行: paid off -> 0
df.iloc[[0, 45341]]

### 決定木の作成

決定木の深さは max_depth=2

In [ ]:
# 説明変数、目的変数を作成
Y_label = 'outcome'
X = df.drop([Y_label], axis=1)
Y = df[Y_label]

# 決定木を作成
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion='entropy', max_depth=2)
model.fit(X, Y)

### 精度の計算・表示

max_depth=2 のため、精度は高くない

- ダミー変数化したため pos_label で正例ラベルの指定が不要

In [ ]:
# 正確度、適合度、再現率（感度）、AUCの算出
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
Y_predict = model.predict(X)
Y_proba = model.predict_proba(X)
fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
print('正確度={:.3f}, 適合度={:.3f}, 再現率={:.3f}, AUC={:.3f}'.format(
    accuracy_score(Y, Y_predict), precision_score(Y, Y_predict),
    recall_score(Y, Y_predict), auc(fpr, tpr)))

### 決定木の描画

In [ ]:
model.classes_

ダミー変数化により model.classes_ が文字列ではなくなり、数値となったため```class_names=model.classes_```での分類クラス名の設定がエラーになるため、分類クラス名の文字列を指定する必要

In [ ]:
# 作成した決定木の描画
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(12, 10))
# plot_tree(model, feature_names=X.columns, class_names=model.classes_, filled=True)
# - class_names=model.classes_ のままだとエラーになるため、文字列を指定
plot_tree(model, feature_names=X.columns, class_names=['paid off', 'default'], filled=True)
plt.show()

### 情報利得の総和

説明変数の重要度

In [ ]:
# 情報利得の総和
pd.DataFrame(model.feature_importances_, index=X.columns, columns=['gain']).\
    sort_values('gain', ascending=False)

<img src='slides/8_25.png'>

In [ ]:
# borrower_score, payment_inc_ratio の散布図
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(x=df.borrower_score, y=df.payment_inc_ratio, hue=df.outcome)
plt.plot([0.475, 0.475], [0, df.payment_inc_ratio.max()], color='tomato')
plt.plot([0, 0.475], [10.12, 10.12], color='tomato')
plt.plot([0.475, 1.0], [7.744, 7.744], color='tomato')

全般的に outcome=0 (paid off) の青のプロットが目立つ、右下には outcome=1 (default) が無い（paid off の半分くらいは default がある）、など、決定木の結果 (value) とは違う見え方になっている。データ数が多すぎて、重なり多すぎる可能性がある

データをランダムに間引き、データ数を減らした散布図を描画

In [ ]:
# borrower_score, payment_inc_ratio の散布図
import seaborn as sns
import matplotlib.pyplot as plt

# 0.5% をランダム抽出
df_s = df.sample(frac=0.005)

sns.scatterplot(x=df_s.borrower_score, y=df_s.payment_inc_ratio, hue=df_s.outcome)
plt.plot([0.475, 0.475], [0, df_s.payment_inc_ratio.max()], color='tomato')
plt.plot([0, 0.475], [10.12, 10.12], color='tomato')
plt.plot([0.475, 1.0], [7.744, 7.744], color='tomato')

<img src="slides/7.png">

### 決定木の深さの変更

決定木の深さを max_depth=6 として決定木を作成してみる。

In [ ]:
# 決定木を作成
model = DecisionTreeClassifier(criterion='entropy', max_depth=6)
model.fit(X, Y)

# 正確度、適合度、再現率（感度）、AUCの算出
Y_predict = model.predict(X)
Y_proba = model.predict_proba(X)
fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
print('正確度={:.3f}, 適合度={:.3f}, 再現率={:.3f}, AUC={:.3f}'.format(
    accuracy_score(Y, Y_predict), precision_score(Y, Y_predict),
    recall_score(Y, Y_predict), auc(fpr, tpr)))

# 決定木の描画
plt.figure(figsize=(12, 10))
plot_tree(model, feature_names=X.columns, class_names=['paid off', 'default'], filled=True)
plt.show()

max_depth=6 の決定木は複雑だが、精度はあまり上がらない。